In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import math
import time 
import random

In [33]:
url = 'https://movie.douban.com/subject/34779776/comments?start=0&limit=20&status=P&sort=new_score'
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
}
response = requests.get(url=url, headers=headers)

In [34]:
response

<Response [200]>

In [35]:
html = response.text
# parse html
soup = BeautifulSoup(html, 'html.parser')

In [36]:
soup

<!DOCTYPE html>

<html class="ua-windows ua-webkit" lang="zh-CN">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="webkit" name="renderer"/>
<meta content="always" name="referrer"/>
<meta content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" name="google-site-verification">
<title>

苍兰诀 短评
</title>
<meta content="cZdR4xxR7RxmM4zE" name="baidu-site-verification">
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Sun, 6 Mar 2005 01:00:00 GMT" http-equiv="Expires"/>
<meta content="苍兰诀,影讯,排片,放映时间,电影票价,在线购票" name="keywords">
<meta content="苍兰诀短评" name="description">
<meta content="format=html5; url=https://m.douban.com/movie/subject/34779776/comments" name="mobile-agent">
<script data-cfg-autoload="false" src="https://img9.doubanio.com/f/shire/2c0c1c6b83f9a457b0f38c38a32fc43a42ec9bad/js/do.js" type="text/javascript"></script>
<link href="https://img9.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-ico

In [37]:
# total number of comments
item_num = soup.find(class_='is-active').find('span').text
item_num = int(re.findall(r'\d+', item_num)[0])
item_num

229114

In [38]:
# comments in the first page
items = soup.find_all('div', attrs={'class': 'comment-item'})
len(items)

20

In [39]:
# the first comment item on the first page
first = items[0]
first

<div class="comment-item" data-cid="3451706098">
<div class="avatar">
<a href="https://www.douban.com/people/135199394/" title="水乡伶人">
<img class="" src="https://img9.doubanio.com/icon/u135199394-8.jpg"/>
</a>
</div>
<div class="comment">
<h3>
<span class="comment-vote">
<span class="votes vote-count">21152</span>
<input type="hidden" value="3451706098">
<a class="j a_show_login" data-id="3451706098" href="javascript:;" onclick="">有用</a>
<!-- 删除短评 -->
</input></span>
<span class="comment-info">
<a class="" href="https://www.douban.com/people/135199394/">水乡伶人</a>
<span>看过</span>
<span class="allstar40 rating" title="推荐"></span>
<span class="comment-time" title="2022-08-09 15:46:33">
                    2022-08-09 15:46:33
                </span>
<span class="comment-location"></span>
</span>
</h3>
<p class="comment-content">
<span class="short">小学生看很幼稚，二十来岁的女大学生看刚刚好🤭</span>
</p>
</div>
</div>

In [40]:
# username
account_title = first.find(class_='avatar').find('a').get('title')
# link to user profile
account_link = first.find(class_='avatar').find('a').get('href')
# number of votes
vote_num = first.find(class_='votes vote-count').text
vote_num = int(vote_num)
# rating
star = first.find(class_='rating').get('title')
# comment time
comment_time = first.find(class_='comment-time').get('title')
# comment location, some do not exist
comment_location = first.find(class_='comment-location').text
# comment string
comment = first.find(class_='comment-content').text.strip()
comment

'小学生看很幼稚，二十来岁的女大学生看刚刚好🤭'

In [41]:
# collect data on the first page
data_dic_list = []
for i in items:
    account_title = i.find(class_='avatar').find('a').get('title')
    account_link = i.find(class_='avatar').find('a').get('href')
    vote_num = i.find(class_='votes vote-count').text
    vote_num = int(vote_num)
    star = i.find(class_='rating').get('title') 
    comment_time = i.find(class_='comment-time').get('title')
    comment_location = i.find(class_='comment-location').text
    comment = i.find(class_='comment-content').text.strip()
    dic = {
        'account_title': account_title,
        'account_link': account_link,
        'vote_num': vote_num,
        'star': star,
        'comment_time': comment_time,
        'comment_location': comment_location,
        'comment': comment
    }
    data_dic_list.append(dic)

In [42]:
pd.DataFrame(data_dic_list)

,account_title,account_link,vote_num,star,comment_time,comment_location,comment
0,水乡伶人,https://www.douban.com/people/135199394/,21152,推荐,2022-08-09 15:46:33,,小学生看很幼稚，二十来岁的女大学生看刚刚好🤭
1,豆瓣会员,https://www.douban.com/people/102823413/,8546,较差,2022-08-07 21:20:18,,受众定位是小学生喜欢的梦幻之七彩琉璃影楼公主过家家风，给我看笑了
2,宁王殿下,https://www.douban.com/people/hoult/,10879,推荐,2022-08-07 21:14:22,,好看好看好看，特效艳丽。情节有趣，笑死我了。会员充得值。
3,山山而川,https://www.douban.com/people/199812512/,7258,还行,2022-08-07 21:54:57,,女主虽然声音很作但是至少咬字清楚（绝对吊打隔壁那个连话都说不清楚的）当然现在的标准降低到这样...
4,寝寝,https://www.douban.com/people/dcx224/,20817,力荐,2022-08-11 23:54:46,,。。很难想象，这是王鹤棣的演技，医学奇迹了。
5,维多利亚,https://www.douban.com/people/loveaxin520/,6536,推荐,2022-08-14 04:34:46,,实话实说，确实土但很上头。王鹤棣碰上虞书欣像是终于开了窍一样，简直是…焕然新生，这才是仙侠届...
6,君越千里,https://www.douban.com/people/122102583/,9027,推荐,2022-08-09 00:17:50,,竟然让我找不到槽点，你说气人不气人！一口气炫了八集，我给八分！什么特么的是惊喜？你可以放心大...
7,庄宝珍,https://www.douban.com/people/82869423/,10525,还行,2022-08-07 21:14:00,,王鹤棣演技进步很大，虞书欣也很灵，两人挺搭，这是可以说的吗？\n小学生看有点幼稚，我这样的大...
8,suicide demo,https://www.douban.com/people/163325886/,17658,力荐,2022-08-15 00:19:29,,我是土狗 我审美低下 我道德败坏 我没有理智
9,猫咪老师,https://www.douban.com/people/55309998/,3460,推荐,2022-08-16 00:20:36,,本剧给我的三大震惊：虞书欣的台词功底，王鹤棣的重生式演技，虽土但爽的合理剧情。


In [43]:
def get_items():
    url = 'https://movie.douban.com/subject/34779776/comments?start=0&limit=20&status=P&sort=new_score'
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
    }
    response = requests.get(url=url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', attrs={'class': 'comment-item'})
    return items

In [44]:
def get_comments(data_dic_list, items):
    for i in items:
        account_title = i.find(class_='avatar').find('a').get('title')
        account_link = i.find(class_='avatar').find('a').get('href')
        vote_num = i.find(class_='votes vote-count').text
        vote_num = int(vote_num)
        star = i.find(class_='rating').get('title') 
        comment_time = i.find(class_='comment-time').get('title')
        comment_location = i.find(class_='comment-location').text
        comment = i.find(class_='comment-content').text.strip()
        dic = {
            'account_title': account_title,
            'account_link': account_link,
            'vote_num': vote_num,
            'star': star,
            'comment_time': comment_time,
            'comment_location': comment_location,
            'comment': comment
        }
        data_dic_list.append(dic)

In [45]:
# total pages
total_pages = math.ceil(item_num / 20)
total_pages 

11456

In [46]:
data_dic_list = []
for i in range(0, total_pages):
    items = get_items()
    get_comments(data_dic_list, items)
    time.sleep(0.2) 
    print(f'{i} is done')

0 is done
1 is done
2 is done
3 is done
4 is done
5 is done
6 is done
7 is done
8 is done
9 is done
10 is done
11 is done
12 is done
13 is done
14 is done
15 is done
16 is done
17 is done
18 is done
19 is done
20 is done
21 is done
22 is done
23 is done
24 is done
25 is done
26 is done
27 is done
28 is done
29 is done
30 is done
31 is done
32 is done
33 is done
34 is done
35 is done
36 is done
37 is done
38 is done
39 is done
40 is done
41 is done
42 is done
43 is done
44 is done
45 is done
46 is done
47 is done
48 is done
49 is done
50 is done
51 is done
52 is done
53 is done
54 is done
55 is done
56 is done
57 is done
58 is done
59 is done
60 is done
61 is done
62 is done
63 is done
64 is done
65 is done
66 is done
67 is done
68 is done
69 is done
70 is done
71 is done
72 is done
73 is done
74 is done
75 is done
76 is done
77 is done
78 is done
79 is done
80 is done
81 is done
82 is done
83 is done
84 is done
85 is done
86 is done
87 is done
88 is done
89 is done
90 is done
91 is don

692 is done
693 is done
694 is done
695 is done
696 is done
697 is done
698 is done
699 is done
700 is done
701 is done
702 is done
703 is done
704 is done
705 is done
706 is done
707 is done
708 is done
709 is done
710 is done
711 is done
712 is done
713 is done
714 is done
715 is done
716 is done
717 is done
718 is done
719 is done
720 is done
721 is done
722 is done
723 is done
724 is done
725 is done
726 is done
727 is done
728 is done
729 is done
730 is done
731 is done
732 is done
733 is done
734 is done
735 is done
736 is done
737 is done
738 is done
739 is done
740 is done
741 is done
742 is done
743 is done
744 is done
745 is done
746 is done
747 is done
748 is done
749 is done
750 is done
751 is done
752 is done
753 is done
754 is done
755 is done
756 is done
757 is done
758 is done
759 is done
760 is done
761 is done
762 is done
763 is done
764 is done
765 is done
766 is done
767 is done
768 is done
769 is done
770 is done
771 is done
772 is done
773 is done
774 is done
775 

1346 is done
1347 is done
1348 is done
1349 is done
1350 is done
1351 is done
1352 is done
1353 is done
1354 is done
1355 is done
1356 is done
1357 is done
1358 is done
1359 is done
1360 is done
1361 is done
1362 is done
1363 is done
1364 is done
1365 is done
1366 is done
1367 is done
1368 is done
1369 is done
1370 is done
1371 is done
1372 is done
1373 is done
1374 is done
1375 is done
1376 is done
1377 is done
1378 is done
1379 is done
1380 is done
1381 is done
1382 is done
1383 is done
1384 is done
1385 is done
1386 is done
1387 is done
1388 is done
1389 is done
1390 is done
1391 is done
1392 is done
1393 is done
1394 is done
1395 is done
1396 is done
1397 is done
1398 is done
1399 is done
1400 is done
1401 is done
1402 is done
1403 is done
1404 is done
1405 is done
1406 is done
1407 is done
1408 is done
1409 is done
1410 is done
1411 is done
1412 is done
1413 is done
1414 is done
1415 is done
1416 is done
1417 is done
1418 is done
1419 is done
1420 is done
1421 is done
1422 is done

1977 is done
1978 is done
1979 is done
1980 is done
1981 is done
1982 is done
1983 is done
1984 is done
1985 is done
1986 is done
1987 is done
1988 is done
1989 is done
1990 is done
1991 is done
1992 is done
1993 is done
1994 is done
1995 is done
1996 is done
1997 is done
1998 is done
1999 is done
2000 is done
2001 is done
2002 is done
2003 is done
2004 is done
2005 is done
2006 is done
2007 is done
2008 is done
2009 is done
2010 is done
2011 is done
2012 is done
2013 is done
2014 is done
2015 is done
2016 is done
2017 is done
2018 is done
2019 is done
2020 is done
2021 is done
2022 is done
2023 is done
2024 is done
2025 is done
2026 is done
2027 is done
2028 is done
2029 is done
2030 is done
2031 is done
2032 is done
2033 is done
2034 is done
2035 is done
2036 is done
2037 is done
2038 is done
2039 is done
2040 is done
2041 is done
2042 is done
2043 is done
2044 is done
2045 is done
2046 is done
2047 is done
2048 is done
2049 is done
2050 is done
2051 is done
2052 is done
2053 is done

2608 is done
2609 is done
2610 is done
2611 is done
2612 is done
2613 is done
2614 is done
2615 is done
2616 is done
2617 is done
2618 is done
2619 is done
2620 is done
2621 is done
2622 is done
2623 is done
2624 is done
2625 is done
2626 is done
2627 is done
2628 is done
2629 is done
2630 is done
2631 is done
2632 is done
2633 is done
2634 is done
2635 is done
2636 is done
2637 is done
2638 is done
2639 is done
2640 is done
2641 is done
2642 is done
2643 is done
2644 is done
2645 is done
2646 is done
2647 is done
2648 is done
2649 is done
2650 is done
2651 is done
2652 is done
2653 is done
2654 is done
2655 is done
2656 is done
2657 is done
2658 is done
2659 is done
2660 is done
2661 is done
2662 is done
2663 is done
2664 is done
2665 is done
2666 is done
2667 is done
2668 is done
2669 is done
2670 is done
2671 is done
2672 is done
2673 is done
2674 is done
2675 is done
2676 is done
2677 is done
2678 is done
2679 is done
2680 is done
2681 is done
2682 is done
2683 is done
2684 is done

...

8918 is done
8919 is done
8920 is done
8921 is done
8922 is done
8923 is done
8924 is done
8925 is done
8926 is done
8927 is done
8928 is done
8929 is done
8930 is done
8931 is done
8932 is done
8933 is done
8934 is done
8935 is done
8936 is done
8937 is done
8938 is done
8939 is done
8940 is done
8941 is done
8942 is done
8943 is done
8944 is done
8945 is done
8946 is done
8947 is done
8948 is done
8949 is done
8950 is done
8951 is done
8952 is done
8953 is done
8954 is done
8955 is done
8956 is done
8957 is done
8958 is done
8959 is done
8960 is done
8961 is done
8962 is done
8963 is done
8964 is done
8965 is done
8966 is done
8967 is done
8968 is done
8969 is done
8970 is done
8971 is done
8972 is done
8973 is done
8974 is done
8975 is done
8976 is done
8977 is done
8978 is done
8979 is done
8980 is done
8981 is done
8982 is done
8983 is done
8984 is done
8985 is done
8986 is done
8987 is done
8988 is done
8989 is done
8990 is done
8991 is done
8992 is done
8993 is done
8994 is done

9549 is done
9550 is done
9551 is done
9552 is done
9553 is done
9554 is done
9555 is done
9556 is done
9557 is done
9558 is done
9559 is done
9560 is done
9561 is done
9562 is done
9563 is done
9564 is done
9565 is done
9566 is done
9567 is done
9568 is done
9569 is done
9570 is done
9571 is done
9572 is done
9573 is done
9574 is done
9575 is done
9576 is done
9577 is done
9578 is done
9579 is done
9580 is done
9581 is done
9582 is done
9583 is done
9584 is done
9585 is done
9586 is done
9587 is done
9588 is done
9589 is done
9590 is done
9591 is done
9592 is done
9593 is done
9594 is done
9595 is done
9596 is done
9597 is done
9598 is done
9599 is done
9600 is done
9601 is done
9602 is done
9603 is done
9604 is done
9605 is done
9606 is done
9607 is done
9608 is done
9609 is done
9610 is done
9611 is done
9612 is done
9613 is done
9614 is done
9615 is done
9616 is done
9617 is done
9618 is done
9619 is done
9620 is done
9621 is done
9622 is done
9623 is done
9624 is done
9625 is done

10167 is done
10168 is done
10169 is done
10170 is done
10171 is done
10172 is done
10173 is done
10174 is done
10175 is done
10176 is done
10177 is done
10178 is done
10179 is done
10180 is done
10181 is done
10182 is done
10183 is done
10184 is done
10185 is done
10186 is done
10187 is done
10188 is done
10189 is done
10190 is done
10191 is done
10192 is done
10193 is done
10194 is done
10195 is done
10196 is done
10197 is done
10198 is done
10199 is done
10200 is done
10201 is done
10202 is done
10203 is done
10204 is done
10205 is done
10206 is done
10207 is done
10208 is done
10209 is done
10210 is done
10211 is done
10212 is done
10213 is done
10214 is done
10215 is done
10216 is done
10217 is done
10218 is done
10219 is done
10220 is done
10221 is done
10222 is done
10223 is done
10224 is done
10225 is done
10226 is done
10227 is done
10228 is done
10229 is done
10230 is done
10231 is done
10232 is done
10233 is done
10234 is done
10235 is done
10236 is done
10237 is done
10238 

10753 is done
10754 is done
10755 is done
10756 is done
10757 is done
10758 is done
10759 is done
10760 is done
10761 is done
10762 is done
10763 is done
10764 is done
10765 is done
10766 is done
10767 is done
10768 is done
10769 is done
10770 is done
10771 is done
10772 is done
10773 is done
10774 is done
10775 is done
10776 is done
10777 is done
10778 is done
10779 is done
10780 is done
10781 is done
10782 is done
10783 is done
10784 is done
10785 is done
10786 is done
10787 is done
10788 is done
10789 is done
10790 is done
10791 is done
10792 is done
10793 is done
10794 is done
10795 is done
10796 is done
10797 is done
10798 is done
10799 is done
10800 is done
10801 is done
10802 is done
10803 is done
10804 is done
10805 is done
10806 is done
10807 is done
10808 is done
10809 is done
10810 is done
10811 is done
10812 is done
10813 is done
10814 is done
10815 is done
10816 is done
10817 is done
10818 is done
10819 is done
10820 is done
10821 is done
10822 is done
10823 is done
10824 

11339 is done
11340 is done
11341 is done
11342 is done
11343 is done
11344 is done
11345 is done
11346 is done
11347 is done
11348 is done
11349 is done
11350 is done
11351 is done
11352 is done
11353 is done
11354 is done
11355 is done
11356 is done
11357 is done
11358 is done
11359 is done
11360 is done
11361 is done
11362 is done
11363 is done
11364 is done
11365 is done
11366 is done
11367 is done
11368 is done
11369 is done
11370 is done
11371 is done
11372 is done
11373 is done
11374 is done
11375 is done
11376 is done
11377 is done
11378 is done
11379 is done
11380 is done
11381 is done
11382 is done
11383 is done
11384 is done
11385 is done
11386 is done
11387 is done
11388 is done
11389 is done
11390 is done
11391 is done
11392 is done
11393 is done
11394 is done
11395 is done
11396 is done
11397 is done
11398 is done
11399 is done
11400 is done
11401 is done
11402 is done
11403 is done
11404 is done
11405 is done
11406 is done
11407 is done
11408 is done
11409 is done
11410 

In [47]:
df = pd.DataFrame(data_dic_list)
df.to_csv('canglangjue_data.csv', index=False)